In [ ]:
#安装Environment
!pip install langchain chromadb sentence-transformers pypdf openai tiktoken langchain_community langchain_text_splitters langchain_openai

In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import OpenAI  # 或 HuggingFaceHub
from langchain.prompts import PromptTemplate

# 1. Load本地知識庫 (Replace with your own knowledge base)
loader = TextLoader("./knowledge.txt")  # 假設有一個名為 knowledge.txt 的文件
documents = loader.load()

custom_prompt = PromptTemplate(
    template="""請基於以下上下文回答問題。如果上下文不包含答案，請運用你的常識回答：
    
    上下文：
    {context}
    問題：{question}
    答案：""",
    input_variables=["context", "question"]
)


# 2. 拆分文本為段落
# 使用 RecursiveCharacterTextSplitter 來將文檔拆分為更小的段落，以便於向量化和檢索
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,  # 每段文本長度
    chunk_overlap=50  # 段落重疊長度
)
texts = text_splitter.split_documents(documents)

# 3. 創建向量數據庫
# 使用 SentenceTransformerEmbeddings 將文本轉換為向量
embedding_model = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
vector_db = Chroma.from_documents(
    documents=texts,
    embedding=embedding_model,
    persist_directory="./rag_db"  # 本地儲存的Path
)

# 4. Setup檢索器
# 使用最大邊際相關性搜索 (MMR) 或相似度閾值檢索
# 這裡選擇 MMR 搜索，返回最相關的片段
retriever = vector_db.as_retriever(
    search_type="mmr",  # 最大边际相关性搜索
    search_kwargs={
        "k": 5  # 返回最相關的5個片段
    }
)

# 5. 配置 LLM
# 這裡可以選擇使用 OpenAI 的 GPT 模型或開源
## 選項A: 使用OpenAI
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
        api_key= '你的API密鑰',  # 替換為你的OpenAI API密鑰
        temperature=0.7,  # 控制生成文本的隨機性
        model = '模型名稱',  # 替換為你想使用的模型名稱，如 'gpt-3.5-turbo'
        max_tokens=1000,  # 控制生成文本的最大長度
        base_url = 'base_url'  # 替換為你的OpenAI API基礎URL，如果使用自定義部署
    )

## 選項B: 使用 HuggingFace 的 LLM
# from langchain_community.llms import HuggingFacePipeline
# from langchain_community.llms import HuggingFaceHub
# llm = HuggingFaceHub(repo_id="meta-llama/Llama-2-7b-chat-hf")


# 6. 創建RAG Chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",  # 簡單拼接檢索結果
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": custom_prompt}  # 應用自定義Prompt
)

In [ ]:
# 7. 提問測試
query = "說明一下地球的基本信息"
result = qa_chain.invoke({"query": query})
print(f"答案：{result['result']}")
print("\nDoc來源：")
for doc in result['source_documents']:
    print(f"- {doc.metadata['source']}")